# Etherscan API



The purpose of this notebook is to pull data from "jaredfromsubway.eth" wallet.

Address on Etherscan: https://etherscan.io/address/0xae2fc483527b8ef99eb5d9b44875f005ba1fae13

## Functions 
* make_api_url
  * Creates API URL 
* get_account_balance
  * Get ETH balance from an address
* get_transactions
  * Get normal and internal transactions from an address
  * Plots the transaction history on a graph


## Resources
* API Documentation:
  * https://docs.etherscan.io/api-endpoints/accounts
* API Walkthrough: 
  * https://www.youtube.com/watch?v=x5FHbr0Em5A
* Etherscan Addresses:
  * https://etherscan.io/accounts 
* 'Normal' vs 'Internal' transactions: 
  * https://www.geeksforgeeks.org/normal-transactions-vs-internal-transactions-in-etherscan/


# Setup

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import web3
from requests import get
from matplotlib import pyplot as plt
from datetime import datetime

In [3]:
API_KEY = "295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH"  # Using API key from RUSSELLTJH
BASE_URL = "https://api.etherscan.io/api" # Base URL for API
ETHER_VALUE = 10 ** 18 # Value to convert Wei to ETH

jaredfromsubway = "0xae2Fc483527B8EF99EB5D9B44875F005ba1FaE13"

# Functions

In [4]:
# Function: create API URL with module, action and address
def make_api_url(module, action, address, **kwargs):
  url = BASE_URL + f"?module={module}&action={action}&address={address}&apikey={API_KEY}"
  # Using an f-string, we can quickly reference variables in a string
  # **kwargs allows us to accept an unlimited amount of keyword arguments, that we will get as a dictionary
  for key, value in kwargs.items():
    url += f"&{key}={value}"
  return url
# ==================================================

In [5]:
# Function: get balance of an address
def get_account_balance(address):
  balance_url = make_api_url("account", "balance", address, tag="latest")
  response = get(balance_url)
  data = response.json()

  value = int(data["result"]) / ETHER_VALUE  # balance of ETH in wei
  return value
# ==================================================

# Sample Request
#   https://api.etherscan.io/api
#     ?module=account
#     &action=balance
#     &address=0xde0b295669a9fd93d5f28d9ec85e40f4cb697bae
#     &tag=latest
#     &apikey=295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH

# Sample Result Response
#   "40891626854930000000000"

In [6]:
# Function: get transactions of address
def get_transactions_df(address):
  # Get 'Normal' transactions
  transactions_url = make_api_url(
      "account", 
      "txlist", 
      address, 
      startblock=0,       # Transactions starting from this block
      endblock=99999999,  # Transactions until this block
      page=1,             # Each page can only show 10,000 transactions
      offset=1000,        # How many transactions that you want to get
      sort="asc")         # Ascending = first transaction to latest transaction
  response = get(transactions_url)
  data = response.json()['result']

  df = pd.DataFrame(data)
  return df
# ==================================================

# Sample Request
#   https://api.etherscan.io/api
#     ?module=account
#     &action=txlist
#     &address=0xc5102fE9359FD9a28f877a67E36B0F050d81a3CC
#     &startblock=0
#     &endblock=99999999
#     &page=1
#     &offset=10
#     &sort=asc
#     &apikey=295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH

# Sample Result Response
  # "blockNumber":"14923692",
  # "timeStamp":"1654646570",
  # "hash":"0xaa45b4858ba44230a5fce5a29570a5dec2bf1f0ba95bacdec4fe8f2c4fa99338",
  # "nonce":"7",
  # "blockHash":"0x8df71a12a8c06b36c06c26bf6248857dd2a2b75b6edbb4e33e9477078897b282",
  # "transactionIndex":"27",
  # "from":"0x9aa99c23f67c81701c772b106b4f83f6e858dd2e",
  # "to":"0xc5102fe9359fd9a28f877a67e36b0f050d81a3cc",
  # "value":"0",
  # "gas":"6000000",
  # "gasPrice":"125521409858",
  # "isError":"0",
  # "txreceipt_status":"1",
  # "input":"0xa9059cbb000000000000000000000000313143c4088a47c469d06fe3fa5fd4196be6a4d600000000000000000000000000000000000000000003b8e97d229a2d54800000",
  # "contractAddress":"",
  # "cumulativeGasUsed":"1977481",
  # "gasUsed":"57168",
  # "confirmations":"122471",
  # "methodId":"0xa9059cbb",
  # "functionName":"transfer(address _to, uint256 _value)"

In [7]:
# Function: get transactions of address
def get_internal_transactions_df(address):
# Get 'Internal' transactions
  internal_tx_url = make_api_url(
      "account", 
      "txlistinternal", 
      address, 
      startblock=0,       # Transactions starting from this block
      endblock=99999999,  # Transactions until this block
      page=1,             # Each page can only show 10,000 transactions
      offset=1000,        # How many transactions that you want to get
      sort="asc")         # Ascending = first transaction to latest transaction
  # Note: Internal transaction API does not include gasPrice information
  response = get(internal_tx_url)
  data = response.json()['result']

  df = pd.DataFrame(data)
  return df
# ================================================== 

# Sample Request
#     https://api.etherscan.io/api
#         ?module=account
#         &action=txlistinternal
#         &address=0x2c1ba59d6f58433fb1eaee7d20b26ed83bda51a3
#         &startblock=0
#         &endblock=2702578
#         &page=1
#         &offset=10
#         &sort=asc
#         &apikey=295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH

# Sample Result Response
#     "blockNumber":"2535368",
#     "timeStamp":"1477837690",
#     "hash":"0x8a1a9989bda84f80143181a68bc137ecefa64d0d4ebde45dd94fc0cf49e70cb6",
#     "from":"0x20d42f2e99a421147acf198d775395cac2e8b03d",
#     "to":"",
#     "value":"0",
#     "contractAddress":"0x2c1ba59d6f58433fb1eaee7d20b26ed83bda51a3",
#     "input":"",
#     "type":"create",
#     "gas":"254791",
#     "gasUsed":"46750",
#     "traceId":"0",
#     "isError":"0",
#     "errCode":""

In [8]:
# Function: get ERC20 transfer events
def get_ERC20_transfers_df(address):
  tokentransfer_url = make_api_url(
      "account", 
      "tokentx", 
      address, 
      # contractaddress: optional filtering by contract address
      startblock=0,       # Transactions starting from this block
      endblock=99999999,  # Transactions until this block
      page=1,             # Each page can only show 10,000 transactions
      offset=1000,        # Number of transactions displayed per page
      sort="asc")         # Ascending = first transaction to latest transaction
  response = get(tokentransfer_url)
  data = response.json()['result']

  df = pd.DataFrame(data)
  return df

# Sample Request
#   https://api.etherscan.io/api
#     ?module=account
#     &action=tokentx
#     &contractaddress=0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
#     &address=0x4e83362442b8d1bec281594cea3050c8eb01311c
#     &page=1
#     &offset=100
#     &startblock=0
#     &endblock=27025780
#     &sort=asc
#     &apikey=295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH

# Sample Result Response
#   "blockNumber":"4730207",
#   "timeStamp":"1513240363",
#   "hash":"0xe8c208398bd5ae8e4c237658580db56a2a94dfa0ca382c99b776fa6e7d31d5b4",
#   "nonce":"406",
#   "blockHash":"0x022c5e6a3d2487a8ccf8946a2ffb74938bf8e5c8a3f6d91b41c56378a96b5c37",
#   "from":"0x642ae78fafbb8032da552d619ad43f1d81e4dd7c",
#   "contractAddress":"0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2",
#   "to":"0x4e83362442b8d1bec281594cea3050c8eb01311c",
#   "value":"5901522149285533025181",
#   "tokenName":"Maker",
#   "tokenSymbol":"MKR",
#   "tokenDecimal":"18",
#   "transactionIndex":"81",
#   "gas":"940000",
#   "gasPrice":"32010000000",
#   "gasUsed":"77759",
#   "cumulativeGasUsed":"2523379",
#   "input":"deprecated",
#   "confirmations":"7968350"

In [9]:
# Function: get ERC721 transfer events
def get_ERC721_transfers_df(address):
  tokentransfer_url = make_api_url(
      "account", 
      "tokentx", 
      address, 
      # contractaddress: optional filtering by contract address
      startblock=0,       # Transactions starting from this block
      endblock=99999999,  # Transactions until this block
      page=1,             # Each page can only show 10,000 transactions
      offset=1000,        # Number of transactions displayed per page
      sort="asc")         # Ascending = first transaction to latest transaction
  response = get(tokentransfer_url)
  data = response.json()['result']

  df = pd.DataFrame(data)
  return df

# Sample Request
#   https://api.etherscan.io/api
#     ?module=account
#     &action=tokentx
#     &contractaddress=0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
#     &address=0x4e83362442b8d1bec281594cea3050c8eb01311c
#     &page=1
#     &offset=100
#     &startblock=0
#     &endblock=27025780
#     &sort=asc
#     &apikey=295NM4HRBDB3RCCBGZCBC8Z6ED6SX6DZIH

# Sample Result Response
#   "blockNumber":"4730207",
#   "timeStamp":"1513240363",
#   "hash":"0xe8c208398bd5ae8e4c237658580db56a2a94dfa0ca382c99b776fa6e7d31d5b4",
#   "nonce":"406",
#   "blockHash":"0x022c5e6a3d2487a8ccf8946a2ffb74938bf8e5c8a3f6d91b41c56378a96b5c37",
#   "from":"0x642ae78fafbb8032da552d619ad43f1d81e4dd7c",
#   "contractAddress":"0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2",
#   "to":"0x4e83362442b8d1bec281594cea3050c8eb01311c",
#   "value":"5901522149285533025181",
#   "tokenName":"Maker",
#   "tokenSymbol":"MKR",
#   "tokenDecimal":"18",
#   "transactionIndex":"81",
#   "gas":"940000",
#   "gasPrice":"32010000000",
#   "gasUsed":"77759",
#   "cumulativeGasUsed":"2523379",
#   "input":"deprecated",
#   "confirmations":"7968350"

# Data Pull

In [10]:
jared_normal = get_transactions_df(jaredfromsubway)
jared_internal = get_internal_transactions_df(jaredfromsubway)
jared_ERC20 = get_ERC20_transfers_df(jaredfromsubway)
jared_ERC721 = get_ERC721_transfers_df(jaredfromsubway)


# Data Processing
- Determine if the Transactions are coming IN or going OUT
- Determine the address that the wallet interacted with

In [11]:
# Function to determine if a transaction is incoming or outgoing
def get_transaction_type(row):
    wallet_address = jaredfromsubway.lower()
    if row["from"] == wallet_address:
        return "OUT", row["to"]
    elif row["to"] == wallet_address:
        return "IN", row["from"]
    else:
        return "UNKOWN", "???"


# Apply the function to each row and create a new column
def create_transaction_type_column(data):
    output = data.apply(get_transaction_type, axis=1)
    data[["transaction_type", "interactionWith"]] = pd.DataFrame(output.values.tolist(), index=data.index)
    return data

jared_normal = create_transaction_type_column(jared_normal)
jared_internal = create_transaction_type_column(jared_internal)
jared_ERC20 = create_transaction_type_column(jared_ERC20)
jared_ERC721 = create_transaction_type_column(jared_ERC721)

In [12]:
jared_normal.head()

,blockNumber,timeStamp,hash,nonce,blockHash,transactionIndex,from,to,value,gas,...,txreceipt_status,input,contractAddress,cumulativeGasUsed,gasUsed,confirmations,methodId,functionName,transaction_type,interactionWith
0,16733027,1677664667,0x5b5d7168a89bf036b3e2a2b7ce130f5437fd6a60bb4d...,27622,0x7241debbd966e1d55f13d4bf772e17fbfb81c33dd6c7...,109,0x4d521577f820525964c392352bb220482f1aa63b,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,10000000000000000,21000,...,1,0x,,10737606,21000,559906,0x,,IN,0x4d521577f820525964c392352bb220482f1aa63b
1,16733089,1677665411,0xa12cab7048bdc47f8fd0394909a35e852593c7e3c72a...,0,0xd8310c258edd139b55fefb9d497acf346febf58e0e88...,202,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0xa7003527af20001c000037a90051b19ce31eed36,5000000000000000,21000,...,0,0x,,10492704,21000,559844,0x,,OUT,0xa7003527af20001c000037a90051b19ce31eed36
2,16733194,1677666683,0x87093f402a4f2754dbab65ffcbf6381e64541f10bc5b...,1,0x8d6b738b21419fd7d64ad0eda4329c6a016f8e3d865d...,213,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0xc10029a6017d184d1f368d32b71f00f33f31773a,0,50000,...,1,0x5c,,14844376,26071,559739,0x5c,,OUT,0xc10029a6017d184d1f368d32b71f00f33f31773a
3,16733562,1677671171,0xc967320e0d6a4f42bb2093f7661ab7b94d9fee503623...,2,0x256f5970065844bf680dcbeecf29ab7ded408b839c19...,100,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,232830,60000,...,1,0x57,,10070846,44515,559371,0x57,,OUT,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
4,16733585,1677671447,0xf2c392aa0c41fb92e52f96a3b8075889397999e1bcd8...,57945,0x2676e9ee0f4b013e4547b2335fe47e0c0cb892210efe...,105,0x00000027f490acee7f11ab5fdd47209d6422c5a7,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,1000000000000000000,21000,...,1,0x,,7587072,21000,559348,0x,,IN,0x00000027f490acee7f11ab5fdd47209d6422c5a7


In [13]:
jared_internal.head()

,blockNumber,timeStamp,hash,from,to,value,contractAddress,input,type,gas,gasUsed,traceId,isError,errCode,transaction_type,interactionWith
0,16733194,1677666683,0x87093f402a4f2754dbab65ffcbf6381e64541f10bc5b...,0xc10029a6017d184d1f368d32b71f00f33f31773a,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0,,,self-destruct,0,0,0_1,0,,IN,0xc10029a6017d184d1f368d32b71f00f33f31773a
1,16733562,1677671171,0xc967320e0d6a4f42bb2093f7661ab7b94d9fee503623...,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,999997235760510,,,call,15296,0,0_1,0,,IN,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
2,16736637,1677708491,0x640d7cab6b5bb4ce5d38ef1e913c1f3c03339c61506f...,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,5000000758880544463,,,call,104087,0,0_1,0,,IN,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
3,16737165,1677714899,0xb149f8749ed13db8030714a312c7ab1a67a78a97caa0...,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,5000000162491712279,,,call,104087,0,0_1,0,,IN,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
4,16737676,1677721187,0xb1998a9bafc57fcfb67eb4ec2c71f8d11b8d78c83f53...,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,5000000130788861004,,,call,104087,0,0_1,0,,IN,0x6b75d8af000000e20b7a7ddf000ba900b4009a80


In [14]:
jared_ERC20.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,transaction_type,interactionWith
0,16771885,1678136987,0x6164e958dc871129d28ce8caac061db313e563c33882...,968,0x0549e879d4793e06c209c3ce38b7515610bb7fdcd697...,0x56f1471a5ea95ddee90f91a228f3e145cfc6f5d4,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,5862000000,USD Coin,...,6,74,99262,51733802759,65637,6978275,deprecated,521048,IN,0x56f1471a5ea95ddee90f91a228f3e145cfc6f5d4
1,16781387,1678252403,0x61e0e718bfa8482401b10a303c62d57ded731962edfd...,27392,0x05a03ca40ddd59fd03b0bebda5081f60475fc7b38931...,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,5862000000,USD Coin,...,6,64,100000,19448824569,43665,9946619,deprecated,511546,OUT,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
2,16837888,1678939067,0x0a4f02ac3bff7f137731ad2c014d63c5de4b3a75b52a...,130,0xb40df2b381ec1e03ba513a58db6407aea51555007a76...,0x6c7a7b1a69dbc211ccc0cba946919794f1fb0b25,0xb7e55e29fc4fe29680fb5b69bcf16a54961326d8,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0,MoonDetector,...,9,32,171940,20971115485,114627,4748558,deprecated,455045,IN,0x6c7a7b1a69dbc211ccc0cba946919794f1fb0b25
3,16957306,1680388619,0x8473ce3f97c8f2e8d5b079e6739623a201013943885d...,49,0x2986ccbbf6523e4bc4b641aa423542093290cafdbd14...,0x4f39af1fc5587318bcd92adf6ec933db5a59a025,0x320a11ec568ee993fe8a398723531211177445db,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,199999000000000,EthQuake,...,9,96,149590,16542205854,90051,8541979,deprecated,335627,IN,0x4f39af1fc5587318bcd92adf6ec933db5a59a025
4,17075356,1681842851,0xed85d1c81b17882f76c767cf54dc3ac75bb2692c5bf8...,947,0xdb7f9a3ac4ceff09fcfe0e25b753d1f41b7eb833dc4b...,0xf44c1aac5ec121398aa87a2c1194c388318f595e,0x6b89b97169a797d94f057f4a0b01e2ca303155e4,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,1000000000000000000,Chad Coin,...,18,135,78232,69795166455,52155,10446798,deprecated,217577,IN,0xf44c1aac5ec121398aa87a2c1194c388318f595e


In [15]:
jared_ERC721.head()

,blockNumber,timeStamp,hash,nonce,blockHash,from,contractAddress,to,value,tokenName,...,tokenDecimal,transactionIndex,gas,gasPrice,gasUsed,cumulativeGasUsed,input,confirmations,transaction_type,interactionWith
0,16771885,1678136987,0x6164e958dc871129d28ce8caac061db313e563c33882...,968,0x0549e879d4793e06c209c3ce38b7515610bb7fdcd697...,0x56f1471a5ea95ddee90f91a228f3e145cfc6f5d4,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,5862000000,USD Coin,...,6,74,99262,51733802759,65637,6978275,deprecated,521048,IN,0x56f1471a5ea95ddee90f91a228f3e145cfc6f5d4
1,16781387,1678252403,0x61e0e718bfa8482401b10a303c62d57ded731962edfd...,27392,0x05a03ca40ddd59fd03b0bebda5081f60475fc7b38931...,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0x6b75d8af000000e20b7a7ddf000ba900b4009a80,5862000000,USD Coin,...,6,64,100000,19448824569,43665,9946619,deprecated,511546,OUT,0x6b75d8af000000e20b7a7ddf000ba900b4009a80
2,16837888,1678939067,0x0a4f02ac3bff7f137731ad2c014d63c5de4b3a75b52a...,130,0xb40df2b381ec1e03ba513a58db6407aea51555007a76...,0x6c7a7b1a69dbc211ccc0cba946919794f1fb0b25,0xb7e55e29fc4fe29680fb5b69bcf16a54961326d8,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,0,MoonDetector,...,9,32,171940,20971115485,114627,4748558,deprecated,455045,IN,0x6c7a7b1a69dbc211ccc0cba946919794f1fb0b25
3,16957306,1680388619,0x8473ce3f97c8f2e8d5b079e6739623a201013943885d...,49,0x2986ccbbf6523e4bc4b641aa423542093290cafdbd14...,0x4f39af1fc5587318bcd92adf6ec933db5a59a025,0x320a11ec568ee993fe8a398723531211177445db,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,199999000000000,EthQuake,...,9,96,149590,16542205854,90051,8541979,deprecated,335627,IN,0x4f39af1fc5587318bcd92adf6ec933db5a59a025
4,17075356,1681842851,0xed85d1c81b17882f76c767cf54dc3ac75bb2692c5bf8...,947,0xdb7f9a3ac4ceff09fcfe0e25b753d1f41b7eb833dc4b...,0xf44c1aac5ec121398aa87a2c1194c388318f595e,0x6b89b97169a797d94f057f4a0b01e2ca303155e4,0xae2fc483527b8ef99eb5d9b44875f005ba1fae13,1000000000000000000,Chad Coin,...,18,135,78232,69795166455,52155,10446798,deprecated,217577,IN,0xf44c1aac5ec121398aa87a2c1194c388318f595e


web3.py documentation: https://web3py.readthedocs.io/en/stable/providers.html

In [16]:
def is_contract(address):
  """
  This function uses the Etherscan API to determine if an address is a wallet or a smart contract.

  Args:
    address: The Ethereum address to check.

  Returns:
    True if the address is a smart contract, False if it is a wallet.
  """

  # Create the API URL.
  url = make_api_url("contract", "isContract", address)

  # Make the API request.
  response = get(url)

  # Check the response status code.
  if response.status_code == 200:
    # The address is a smart contract if the response JSON contains the key "code".
    return "code" in response.json()
  else:
    return False

print(is_contract("0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80"))

False


In [43]:
import requests
import json

def is_contract(address):
  """Check if an Ethereum address is a contract.

  Args:
    address: The Ethereum address to check.

  Returns:
    True if the address is a contract, False otherwise.
  """

  url = make_api_url("eth", "getCodeSize", address)
  response = requests.get(url)
  if response.status_code == 200:
    data = json.loads(response.content.decode("utf-8"))
    code_size = int(data["result"])
    if code_size > 0:
      return True
    else:
      return False
  else:
    return False

if __name__ == "__main__":
  address = "0x1234567890abcdef"
  if is_contract(address):
    print("The address is a contract.")
  else:
    print("The address is not a contract.")

is_contract("0x1234567890abcdef") # True
is_contract("0x0000000000000000000000000000000000000001") # False

# print(is_contract("0x6b75d8AF000000e20B7a7DDf000Ba900b4009A80"))

ValueError: invalid literal for int() with base 10: 'Error! Missing Or invalid Module name'

In [58]:
from web3 import Web3, HTTPProvider

w3 = Web3(HTTPProvider("https://mainnet.infura.io/v3/c48543d68919471c82f6fdbff40ef13c"))

def is_contract(address):
  """
  Returns True if the address is a contract, False otherwise.

  Args:
    address: The Ethereum address to check.

  Returns:
    True if the address is a contract, False otherwise.
  """

# Check if the address is a valid Ethereum address.
#   if not web3.isAddress(address):
#     return False

  # Get the code size for the address.
  code_size = web3.eth.get_code(address)

  # If the code size is greater than 0, then the address is a contract.
  return code_size > 0


if __name__ == "__main__":
  # Get the address to check.
  address = "0x..."

  # Check if the address is a contract.
  is_contract = is_contract(address)

  # Print the result.
  if is_contract:
    print("The address is a contract.")
  else:
    print("The address is not a contract.")

AttributeError: module 'web3.eth' has no attribute 'get_code'

In [52]:
pip show web3

Name: web3
Version: 6.4.0
Summary: web3.py
Home-page: https://github.com/ethereum/web3.py
Author: Piper Merriam
Author-email: pipermerriam@gmail.com
License: MIT
Location: c:\users\russell\appdata\local\programs\python\python310\lib\site-packages
Requires: aiohttp, eth-abi, eth-account, eth-hash, eth-typing, eth-utils, hexbytes, jsonschema, lru-dict, protobuf, pywin32, requests, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.
